In [15]:
import pandas as pd

In [18]:
df = pd.read_excel(r"C:\Users\marco\Desktop\GitProjects\app_plagio\Excels\logs-exam.xlsx")
df = df.drop(['Contexto del evento','Dirección IP','Origen'], axis=1)
df = df[df['Nombre del evento'] != 'Reporte de examen visto']


In [19]:
df

,Hora,Nombre completo del usuario,Usuario afectado,Componente,Nombre del evento,Descripción
2,"13/10/24, 14:06:08",Anderson Molina,-,Examen,Módulo de curso visto,The user with id '2' viewed the 'quiz' activit...
3,"11/10/24, 11:18:43",Geraldine Tasayco,-,Examen,Módulo de curso visto,The user with id '52' viewed the 'quiz' activi...
4,"11/10/24, 00:07:57",Patricia Avila,Patricia Avila,Examen,Intento de examen revisado,The user with id '28' has reviewed quiz attemp...
5,"11/10/24, 00:07:53",Patricia Avila,-,Examen,Módulo de curso visto,The user with id '28' viewed the 'quiz' activi...
6,"11/10/24, 00:07:25",Patricia Avila,-,Examen,Módulo de curso visto,The user with id '28' viewed the 'quiz' activi...
...,...,...,...,...,...,...
1662,"9/09/24, 13:39:42",Anderson Molina,-,Examen,Posición creada,The user with id '2' created a new slot with i...
1663,"9/09/24, 13:37:12",Anderson Molina,-,Examen,Página de edición de examen vista,The user with id '2' viewed the edit page for ...
1664,"9/09/24, 13:33:36",Anderson Molina,-,Examen,Página de edición de examen vista,The user with id '2' viewed the edit page for ...
1665,"9/09/24, 13:33:20",Anderson Molina,-,Examen,Módulo de curso visto,The user with id '2' viewed the 'quiz' activit...
